# Solution: Model Relationships

## ⚠️ Try the exercise first!

**Don't look at this solution until you've attempted the exercise yourself!**

## Example Solutions

Here are example solutions for the model relationship exercises:


In [ ]:
# Exercise 1: One-to-Many Relationship Solution

# Setup
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///relationships_exercise.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

print("SQLAlchemy setup complete!")


In [ ]:
# Author Model (Parent - One side)
class Author(Base):
    __tablename__ = 'authors'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    bio = Column(Text)
    
    # One-to-many relationship: One author has many books
    books = relationship("Book", back_populates="author", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Author(name='{self.name}', email='{self.email}')>"

# Book Model (Child - Many side)
class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    isbn = Column(String(20), unique=True)
    published_year = Column(Integer)
    
    # Foreign key to authors table
    author_id = Column(Integer, ForeignKey('authors.id'), nullable=False)
    
    # Many-to-one relationship: Many books belong to one author
    author = relationship("Author", back_populates="books")
    
    def __repr__(self):
        return f"<Book(title='{self.title}', author_id={self.author_id})>"

print("Author and Book models with one-to-many relationship defined!")


In [ ]:
# Create tables and test the relationship
Base.metadata.create_all(engine)
session = Session()

# Create authors
author1 = Author(name='J.K. Rowling', email='jkrowling@example.com', bio='Author of Harry Potter series')
author2 = Author(name='George Orwell', email='gorwell@example.com', bio='Author of 1984 and Animal Farm')

# Add authors to session
session.add_all([author1, author2])
session.commit()

print("Authors created successfully!")

# Create books
book1 = Book(title='Harry Potter and the Philosopher\'s Stone', isbn='978-0747532699', published_year=1997, author_id=author1.id)
book2 = Book(title='Harry Potter and the Chamber of Secrets', isbn='978-0747538493', published_year=1998, author_id=author1.id)
book3 = Book(title='1984', isbn='978-0451524935', published_year=1949, author_id=author2.id)

# Add books to session
session.add_all([book1, book2, book3])
session.commit()

print("Books created successfully!")

# Test the relationship
print(f"\nAuthor: {author1}")
print(f"Author's books: {author1.books}")

print(f"\nBook: {book1}")
print(f"Book's author: {book1.author}")


In [ ]:
# Exercise 2: Many-to-Many Relationship Solution

# Create association table
from sqlalchemy import Table

# Association table for Student-Course many-to-many relationship
enrollments = Table('enrollments', Base.metadata,
    Column('student_id', Integer, ForeignKey('students.id'), primary_key=True),
    Column('course_id', Integer, ForeignKey('courses.id'), primary_key=True),
    Column('enrolled_at', DateTime, default=datetime.utcnow),
    Column('grade', String(2))  # Additional data in association table
)

print("Association table created!")


In [ ]:
# Student Model
class Student(Base):
    __tablename__ = 'students'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    major = Column(String(50))
    
    # Many-to-many relationship with courses
    courses = relationship("Course", secondary=enrollments, back_populates="students")
    
    def __repr__(self):
        return f"<Student(name='{self.name}', major='{self.major}')>"

# Course Model
class Course(Base):
    __tablename__ = 'courses'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    description = Column(Text)
    credits = Column(Integer, default=3)
    department = Column(String(50))
    
    # Many-to-many relationship with students
    students = relationship("Student", secondary=enrollments, back_populates="courses")
    
    def __repr__(self):
        return f"<Course(title='{self.title}', credits={self.credits})>"

print("Student and Course models with many-to-many relationship defined!")


In [ ]:
# Create tables and test many-to-many relationship
Base.metadata.create_all(engine)

# Create students
student1 = Student(name='Alice Johnson', email='alice@university.edu', major='Computer Science')
student2 = Student(name='Bob Smith', email='bob@university.edu', major='Mathematics')
student3 = Student(name='Charlie Brown', email='charlie@university.edu', major='Physics')

# Create courses
course1 = Course(title='Python Programming', description='Learn Python fundamentals', credits=4, department='CS')
course2 = Course(title='Database Design', description='Database concepts and SQL', credits=3, department='CS')
course3 = Course(title='Calculus I', description='Introduction to calculus', credits=4, department='MATH')

# Add to session
session.add_all([student1, student2, student3, course1, course2, course3])
session.commit()

print("Students and courses created successfully!")

# Create many-to-many relationships
student1.courses.append(course1)  # Alice enrolls in Python
student1.courses.append(course2)  # Alice enrolls in Database Design
student2.courses.append(course1)  # Bob enrolls in Python
student2.courses.append(course3)  # Bob enrolls in Calculus
student3.courses.append(course2)  # Charlie enrolls in Database Design
student3.courses.append(course3)  # Charlie enrolls in Calculus

session.commit()
print("Enrollments created successfully!")

# Test the relationship
print(f"\nStudent: {student1}")
print(f"Student's courses: {student1.courses}")

print(f"\nCourse: {course1}")
print(f"Course's students: {course1.students}")

# Close session
session.close()


## 🎯 Key Learning Points

1. **One-to-Many Relationships**: Use `ForeignKey()` in the child model and `relationship()` in both models
2. **Many-to-Many Relationships**: Use `Table()` for association table and `secondary` parameter in `relationship()`
3. **Bidirectional Relationships**: Use `back_populates` to create two-way relationships
4. **Cascade Options**: Use `cascade="all, delete-orphan"` for automatic cleanup
5. **Association Tables**: Can include additional columns beyond foreign keys

## 💡 Alternative Solutions

There are often multiple ways to solve the same problem:

- **Model Names**: Author/Book, Company/Employee, Category/Product - choose what fits your scenario
- **Relationship Names**: `books`, `author`, `employees`, `company` - use descriptive names
- **Association Table Names**: `enrollments`, `tags`, `categories` - be descriptive
- **Additional Columns**: Add timestamps, status, grades to association tables

## 🚀 Next Steps

You've mastered model relationships! You're ready to move on to:
- Advanced querying with joins
- Relationship loading strategies
- Database migrations
- Performance optimization
- Complex relationship patterns

The important thing is that your relationships work correctly and your data integrity is maintained!
